# Natural Language Processing ( NLP)

## Project 1
## Topic Analysis of Review Data

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
reviews0 = pd.read_csv(r"F:\._LEARNING\AI\_6. NLP\project\K8 Reviews v0.2.csv")
reviews0.head()

,sentiment,review
0,1,Good but need updates and improvements
1,0,"Worst mobile i have bought ever, Battery is dr..."
2,1,when I will get my 10% cash back.... its alrea...
3,1,Good
4,0,The worst phone everThey have changed the last...


### lower the sentence 

In [3]:
reviews_lower = [sent.lower() for sent in reviews0.review.values]
reviews_lower[0]


'good but need updates and improvements'

### Prform the Tokenization

In [4]:
# Import the word_tokenize Library 
from nltk.tokenize import word_tokenize 


In [5]:
reviews_token = [word_tokenize(sent) for sent in reviews_lower]
reviews_token[0]


['good', 'but', 'need', 'updates', 'and', 'improvements']

In [6]:
# Import the nltk library
import nltk

### Perform pos_tagging

In [7]:
nltk.pos_tag(reviews_token[0])

[('good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

In [8]:
reviews_tagged = [nltk.pos_tag(tokens) for tokens in reviews_token]

In [9]:
reviews_tagged[0]

[('good', 'JJ'),
 ('but', 'CC'),
 ('need', 'VBP'),
 ('updates', 'NNS'),
 ('and', 'CC'),
 ('improvements', 'NNS')]

In [10]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [11]:
tagged_tuple = nltk.pos_tag(['great'])
tagged_tuple[0]


('great', 'JJ')

### Let’s define the function that will give us only nouns or adjectives

In [12]:
reviews_noun=[]

In [13]:
for sent in reviews_tagged:
    reviews_noun.append([token for token in sent if re.search("NN.*", token[1])])


In [14]:
reviews_noun[0]

[('updates', 'NNS'), ('improvements', 'NNS')]

### Performing Lemmatizing 

In [15]:
# import wordnetlemmatizer library
from nltk import WordNetLemmatizer

In [16]:
lemm = WordNetLemmatizer()

In [17]:
reviews_lemm=[]

for sent in reviews_noun:
    reviews_lemm.append([lemm.lemmatize(word[0]) for word in sent])    

reviews_lemm[0]


['update', 'improvement']

### Remove stopwords and punctuation

In [18]:
# import punctuation and stopwords libraries
from string import punctuation
from nltk.corpus import stopwords


In [19]:
stop_nltk = stopwords.words("english")

In [20]:
stop_updated = stop_nltk + list(punctuation) + ["..."] + [".."]

In [21]:
reviews_sw_removed=[]

In [22]:
for sent in reviews_lemm:
    reviews_sw_removed.append([term for term in sent if term not in stop_updated])


In [23]:
reviews_sw_removed[1]

['mobile',
 'battery',
 'hell',
 'backup',
 'hour',
 'us',
 'idle',
 'discharged.this',
 'lie',
 'amazon',
 'lenove',
 'battery',
 'charger',
 'hour']

### Building Topic modelings with LDA

In [24]:
# Import the necesory libraries
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models import ldamodel


C:\Users\admin\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [25]:
id2word = corpora.Dictionary(reviews_sw_removed)
texts = reviews_sw_removed


In [26]:
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[200])


[(36, 1), (143, 1), (314, 1), (415, 1), (416, 1)]


In [27]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           random_state=42,
                                           passes=10,
                                           per_word_topics=True)


In [28]:
print(lda_model.print_topics())

[(0, '0.138*"mobile" + 0.040*"call" + 0.036*"screen" + 0.031*"feature" + 0.030*"option" + 0.020*"music" + 0.017*"software" + 0.016*"app" + 0.015*"video" + 0.015*"card"'), (1, '0.151*"money" + 0.128*"...." + 0.071*"waste" + 0.056*"value" + 0.046*"glass" + 0.038*"speaker" + 0.024*"gorilla" + 0.022*"set" + 0.022*"ok" + 0.020*"piece"'), (2, '0.216*"note" + 0.113*"k8" + 0.090*"lenovo" + 0.030*"sound" + 0.023*"dolby" + 0.020*"killer" + 0.018*"gallery" + 0.018*"system" + 0.018*"atmos" + 0.018*"excellent"'), (3, '0.078*"phone" + 0.040*"day" + 0.038*"amazon" + 0.035*"service" + 0.034*"issue" + 0.027*"time" + 0.027*"lenovo" + 0.026*"battery" + 0.024*"month" + 0.023*"device"'), (4, '0.280*"product" + 0.176*"problem" + 0.080*"network" + 0.075*"issue" + 0.066*"heating" + 0.021*"jio" + 0.021*"sim" + 0.019*"volta" + 0.010*"connection" + 0.009*"signal"'), (5, '0.093*"heat" + 0.070*"....." + 0.052*"processor" + 0.038*"everything" + 0.038*"budget" + 0.031*"...." + 0.030*"core" + 0.025*"display" + 0.017*

In [29]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews_sw_removed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.475339388396195


In [30]:
lda_mode2 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=42,
                                           passes=10,
                                           per_word_topics=True)


In [31]:
coherence_model_lda = CoherenceModel(model=lda_mode2, texts=reviews_sw_removed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.5442702792781539


### Create a table with the topic name and the top 10 terms in each to present to the business.

In [32]:
x = lda_mode2.show_topics(formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]


In [33]:
for topic,words in topics_words:
    print(str(topic)+ "::"+ str(words))
print()


0::['mobile', 'call', 'screen', 'feature', 'option', 'app', 'card', 'cast', 'image', 'music']
1::['money', 'waste', 'value', 'handset', 'plz', 'headset', 'offer', 'signal', 'facility', 'drainage']
2::['note', 'k8', 'lenovo', 'dolby', 'feature', 'sound', 'system', 'killer', 'atmos', 'excellent']
3::['phone', 'issue', 'day', 'problem', 'lenovo', 'service', 'time', 'month', 'amazon', 'update']
4::['product', 'problem', 'network', 'issue', 'heating', 'amazon', 'sim', 'h', 'return', 'jio']
5::['....', 'battery', 'phone', 'heat', 'hour', 'charge', 'time', '.....', 'charging', 'hr']
6::['price', 'phone', 'feature', 'range', 'budget', 'specification', 'buy', 'smartphone', 'super', 'speaker']
7::['charger', 'hai', 'ho', 'turbo', 'box', 'piece', 'cable', 'bill', 'bhi', 'hi']
8::['delivery', 'superb', 'device', 'headphone', 'thanks', 'earphone', 'set', 'ok', 'pls', 'expectation']
9::['camera', 'battery', 'phone', 'quality', 'performance', 'backup', 'mode', 'issue', 'life', 'day']

